In [5]:
import dash
from dash import dcc, html, dash_table, callback, Output, Input, State
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
from datetime import datetime as dt
from dash.exceptions import PreventUpdate

[Посилання на датасет](https://www.kaggle.com/datasets/heemalichaudhari/adidas-sales-dataset?select=Adidas+US+Sales+Datasets.xlsx)

In [6]:
#Завантажую датасет вручну через основну директорію
df = pd.read_excel('Adidas US Sales Datasets.xlsx')
state_data = df.groupby(['State', 'Sales Method'], as_index=False)['Operating Profit'].sum()
state_data.head()

,State,Sales Method,Operating Profit
0,Alabama,Online,283945.26
1,Alabama,Outlet,8863636.13
2,Alaska,Online,158133.36
3,Alaska,Outlet,4340212.50
4,Arizona,Online,5209599.57


In [7]:
state_name_to_code = {
    'California': 'CA',
    'Texas': 'TX',
    'New York': 'NY',
    'Florida': 'FL',
    'Mississippi': 'MS',
    'Oregon': 'OR',
    'Louisiana': 'LA',
    'Idaho': 'ID',
    'New Mexico': 'NM',
    'Georgia': 'GA',
    'Arkansas': 'AR',
    'Virginia': 'VA',
    'Oklahoma': 'OK',
    'Connecticut': 'CT',
    'Rhode Island': 'RI',
    'Massachusetts': 'MA',
    'Vermont': 'VT',
    'Utah': 'UT',
    'Arizona': 'AZ',
    'New Hampshire': 'NH',
    'Pennsylvania': 'PA',
    'Nevada': 'NV',
    'Alabama': 'AL',
    'Tennessee': 'TN',
    'South Dakota': 'SD',
    'Illinois': 'IL',
    'Colorado': 'CO',
    'New Jersey': 'NJ',
    'Delaware': 'DE',
    'Maryland': 'MD',
    'West Virginia': 'WV',
    'Indiana': 'IN',
    'Wisconsin': 'WI',
    'Iowa': 'IA',
    'North Dakota': 'ND',
    'Michigan': 'MI',
    'Kansas': 'KS',
    'Missouri': 'MO',
    'Minnesota': 'MN',
    'Montana': 'MT',
    'Kentucky': 'KY',
    'Ohio': 'OH',
    'North Carolina': 'NC',
    'South Carolina': 'SC',
    'Nebraska': 'NE',
    'Maine': 'ME',
    'Alaska': 'AK',
    'Hawaii': 'HI',
    'Wyoming': 'WY',
    'Washington': 'WA'   
}

state_data['State Code'] = state_data['State'].map(state_name_to_code)

In [8]:
app = dash.Dash()

app.layout = html.Div([
    html.H4('Adidas sales for 2020-2021 in the USA', style={
        'textAlign': 'center',
        'fontSize': '42px',  
        'fontFamily': 'Helvetica, sans-serif',  
        'color': 'green'}),
    
    html.Div([
        html.Div([
            html.P('Filter by Sales Method:', style={'margin-right': '10px'}),
            dcc.Checklist(
                id='sales-method-filter',
                options=[
                    {'label': 'Online', 'value': 'Online'},
                    {'label': 'Outlet', 'value': 'Outlet'},
                    {'label': 'In-store', 'value': 'In-store'}
                ],
                value=['Online', 'Outlet', 'In-store'],
                style={'display': 'flex', 'flex-direction': 'row'}
            )
        ], style={'display': 'flex', 'align-items': 'center'}),
        
        html.Div([
            html.P('Choose a parameter:'),
            dcc.Dropdown(
                id='parameter',
                options=[
                    {'label': 'Retailer', 'value': 'Retailer'},
                    {'label': 'Product', 'value': 'Product'},
                ],
                value='Retailer',
                clearable=False,
                style={'width': '90%'}
            ),
        ], style={'display': 'flex', 'align-items': 'center', 'margin-top': '50px', 'margin-bottom': '50px'}),
        
        html.Div([
            html.P('Choose a metric:'),
            dcc.Dropdown(
                id='metric',
                options=[
                    {'label': 'Total Sales', 'value': 'Total Sales'},
                    {'label': 'Operating Profit', 'value': 'Operating Profit'},
                    {'label': 'Units Sold', 'value': 'Units Sold'}
                ],
                value='Total Sales',
                clearable=False,
                style={'width': '120%'}
            ),
        ], style={'display': 'flex', 'align-items': 'center', 'margin-top': '50px', 'margin-bottom': '50px'}),
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-evenly'}),
    
    html.Div([
        dcc.Graph(id='map', style={'width': '49%', 'display': 'inline-block'}),
        dcc.Graph(id='graph', style={'width': '49%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    
    html.Div([
        dcc.Graph(id='operating-margin-bar', style={'width': '49%', 'display': 'inline-block'}),
        dcc.Graph(id='price-boxplot', style={'width': '49%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    
    html.Div([
        html.Div([
            html.P('Choose one option for analytics:', style={'display': 'inline-block'}),
            dcc.Checklist(
                id='data-type',
                options=[
                    {'label': 'Product', 'value': 'Product'},
                    {'label': 'Region', 'value': 'Region'},
                    {'label': 'Retailer', 'value': 'Retailer'}
                ],
                value=['Product'],  
                inline=True
            )
        ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'flex-start'}),
        
        html.Div([
            html.P('Choice a region:', style={'display': 'inline-block'}),
            dcc.Checklist(
                id='region-checklist',
                options=[
                    {'label': 'West', 'value': 'West'},
                    {'label': 'Northeast', 'value': 'Northeast'},
                    {'label': 'Midwest', 'value': 'Midwest'},
                    {'label': 'South', 'value': 'South'},
                    {'label': 'Southeast', 'value': 'Southeast'}
                ],
                value=['West', 'Northeast'],
                inline=True
            )
        ], style={'display': 'flex', 'align-items': 'center', 'justify-content': 'flex-end'}),
    ], style={'display': 'flex', 'justify-content': 'space-evenly'}),
])

@app.callback(
    [Output('graph', 'figure'),
     Output('map', 'figure')],
    [Input('parameter', 'value'),
     Input('metric', 'value'),
     Input('sales-method-filter', 'value')]
)
def generate_chart(parameter, metric, sales_method_filter):
    # код для "States operating profit"
    filtered_data = state_data[state_data['Sales Method'].isin(sales_method_filter)]
    
    
    pie_chart = px.pie(df, values=metric, names=parameter, hole=0.3)
    pie_chart.update_layout(title_text='Pie Chart Analysis', title_x=0.5)

    
    choropleth_map = go.Figure(
        go.Choropleth(
            locations=filtered_data['State Code'],
            z=filtered_data['Operating Profit'],
            locationmode='USA-states',
            colorscale='Viridis',
            colorbar_title='Operating Profit'
        )
    )
    choropleth_map.update_layout(
        title_text='States operating profit', title_x=0.5,
        geo_scope='usa',
    )
    return pie_chart, choropleth_map
    pass

@app.callback(
    Output('operating-margin-bar', 'figure'),
    [Input('data-type', 'value')]
)
def update_operating_margin_bar(data_type):
    #код для "Top 10 Products by Operating Margin"
    top_n = 4  
    
    if 'Product' in data_type:
        data_group = 'Product'
        chart_title = 'Top 4 Products by Operating Margin'
    elif 'Region' in data_type:
        data_group = 'Region'
        chart_title = 'Top 4 Regions by Operating Margin'
    else:
        data_group = 'Retailer'
        chart_title = 'Top 4 Retailers by Operating Margin'

    # Групуємо дані та обчислюємо середній показник Operating Margin
    data_margin = df.groupby(data_group)['Operating Margin'].mean().reset_index()

    # сортуємо по DESC і обираємо топ-н
    top_data = data_margin.sort_values('Operating Margin', ascending=False).head(top_n)

    # діаграма
    bar_chart = go.Figure(go.Bar(
        x=top_data[data_group],
        y=top_data['Operating Margin'],
        marker_color='lightgreen'  # колір
    ))

    bar_chart.update_layout(
        title_text=chart_title,
        title_x=0.5,
        xaxis_title=data_group,
        yaxis_title='Operating Margin',
    )

    return bar_chart
@app.callback(
    Output('price-boxplot', 'figure'),
    Input('region-checklist', 'value')
)
def update_boxplot(selected_regions):
    filtered_df = df[df['Region'].isin(selected_regions)]
    fig = px.box(filtered_df, x='Product', y='Price per Unit', color='Region', points='all')
    fig.update_layout(title='Product price box', title_x=0.5)
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=6055)

Після виконання цього коду, аналітичний застосунок можна відкрити за посиланням в браузері: http://127.0.0.1:6055/

Нижче приведені фотографії інтерактивних візуалізацій, які можна змінювати за параметрами:

![Блок 1](part1.png)
![Блок 2](part2.png)